In [3]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error

users = pd.read_table('ml-1m/users.dat',  sep = '::', header = None, names= ['ID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
ratings = pd.read_table('ml-1m/ratings.dat',  sep = '::', header = None, names= ['UserID', 'MovieID', 'Rating', 'Timestamp'])
#movies = pd.read_table('ml-1m/movies.dat',  sep = '::', header = None, names= ['ID', 'Title', 'Genre'])

C:\Users\rolaa\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [4]:
kf = KFold(n_splits = 5)
for train, test in kf.split(ratings):
    print(train, test)

[ 200042  200043  200044 ... 1000206 1000207 1000208] [     0      1      2 ... 200039 200040 200041]
[      0       1       2 ... 1000206 1000207 1000208] [200042 200043 200044 ... 400081 400082 400083]
[      0       1       2 ... 1000206 1000207 1000208] [400084 400085 400086 ... 600123 600124 600125]
[      0       1       2 ... 1000206 1000207 1000208] [600126 600127 600128 ... 800165 800166 800167]
[     0      1      2 ... 800165 800166 800167] [ 800168  800169  800170 ... 1000206 1000207 1000208]


In [18]:
d = 2

utility_matrix = ratings.pivot(index = "UserID", columns = "MovieID", values = "Rating")
a = utility_matrix.stack().mean(skipna = True)

U = np.full([utility_matrix.shape[0],d], np.sqrt(a/d))
V = np.full([d,utility_matrix.shape[1]], np.sqrt(a/d))
matrix = np.dot(U,V)



#print(matrix.shape)
#print(utility_matrix.shape)
#print(utility_matrix)
#print(ratings[ratings["UserID"]==1].sort_values("MovieID"))
#print(utility_matrix.iloc[0,:].dropna())

[[1.33820112 1.33820112]
 [1.33820112 1.33820112]
 [1.33820112 1.33820112]
 ...
 [1.33820112 1.33820112]
 [1.33820112 1.33820112]
 [1.33820112 1.33820112]]
